In [1]:
import os
import pandas as pd
import xlrd
from pathlib import Path
import openpyxl
from openpyxl import load_workbook
import numpy as np
import re

In [2]:
#Identify files with AB 1045 Forms in them, extract worksheets which may contain this form
#names like "Common OP Procedures" or "AB 1045 form"
xlsx_file = Path('SimData', '/home/ygael/Projects/healthcare/data/2019_CDM_chargemaster/Adventist Health Clearlake/106171049_CDM_All_2019.xlsx')
wb_obj = openpyxl.load_workbook(xlsx_file) 

# Read the active sheet:
sheet = wb_obj.active


In [3]:
#Get Chargecode Sheet
#####################################################
datalist=[]
for row in sheet.iter_rows():
    for cell in row:
        datalist.append(cell.value)

#Get column names
col_names = []
for column in sheet.iter_cols(1, sheet.max_column):
    col_names.append(column[0].value)
   
    
#Create a dictionary from the speadsheet

data = {}

for i, row in enumerate(sheet.iter_rows(values_only=True)):
    if i == 0:
        for i in range(len(col_names)):
            data[row[i]] = [col_names[i]]

    else:
        for i in range(len(col_names)):
            data[col_names[i]].append(row[i])

#Create a dataframe from the dictionary
df = pd.DataFrame(data)
df = df.iloc[1:]

df

,Charge Code,Charge Code Description,HCPCS Code,June 2019 Prices
1,1255,ROOM/BED: Intensive Care,None,9350.13
2,7211,ROOM/BED: Med/Surg Private,None,4935.76
3,17509,ROOM/BED: Newborn Routine,None,3172.4
4,21253,ROOM/BED: Med/Surg Semi-Private,None,4390.68
5,27003,ROOM/BED: Obstetrics LDRP Semi-Private,None,4561.25
...,...,...,...,...
8708,9520156,PF PM DEPRESSION SCREENING 95 952,None,0.01
8709,9520511,PF PM DIL RETINAL EYE EXAM W/INTERP 95 952,None,0.01
8710,9520586,PF PM HGB A1C LEVEL < 7.0% 95 952,None,0.01
8711,9520594,PF PM HGB A1C LEVEL 7.0%-9.0% 95 952,None,0.01


In [4]:

    ##Getting the 1045 data
xlsx_file = Path('SimData', '/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster/Alvarado Hospital/106370652_CDM_All_2020.xlsx')
wb_obj = openpyxl.load_workbook(xlsx_file) 

In [5]:
#Function to extract 1045 sheet name

def find1045(filepath):
    #read file
    xlsx_file = Path('SimData', filepath)
    wb_obj = openpyxl.load_workbook(xlsx_file)

    #get sheet names
    sheetnames=wb_obj.sheetnames

    #Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

    #Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            sheetname=(wb_obj[name])
            return(sheetname)
            break


In [6]:
#Get the first part of the data after finding the 1045 sheet (nested)
def getdata(file):
    sheet=find1045(file)
    alphabet=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    found=False
    keywords = ["evaluation", "management"]
    for row in sheet.iter_rows():
        if found == False:
            for cell in row:
                heading = str(cell.value).lower()
                wordsInHeading=heading.split()
                match =  any(item in wordsInHeading for item in keywords)
                if match==True:
                    found = True
                    coord1=cell.coordinate
                    splitcoord=re.split('(\d+)', coord1)
                    alphaindex=(alphabet.index(f'{splitcoord[0]}'))+2
                    alphachar=alphabet[alphaindex]
                    numindex=splitcoord[1]
                    numindex=int(numindex)+3
                    numindex=str(numindex)
                    coord2=f"{alphachar}{numindex}"
                    df=pd.DataFrame(wb_obj["ABS 1045"][f"{coord1}:{coord2}"])
                    values=[]
                    for row in sheet[coord1:coord2]:
                        for i in range(len(row)):
                            values.append(row[i].value)
                    data = np.array(values).reshape((4, 3))
                    df=pd.DataFrame(data=data)
                    df.columns = df.iloc[0]
                    df["Service Type"]=values[0]
                    df.rename(columns={values[0]:"Service"})
                    df.drop(df.index[0])
                    return(df)

In [7]:
getdata(xlsx_file)

,Evaluation & Management Services (CPT Codes 99201-99499),2019 CPT Code,Average Charge,Service Type
0,Evaluation & Management Services (CPT Codes 99...,2019 CPT Code,Average Charge,Evaluation & Management Services (CPT Codes 99...
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,746.55,Evaluation & Management Services (CPT Codes 99...
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,1051.05,Evaluation & Management Services (CPT Codes 99...
3,"Emergency Room Visit, Level 4 (high severity)",99284,1581.3,Evaluation & Management Services (CPT Codes 99...


In [8]:
#Note- change above function to accept lists of lists of (yes, of and of) keywords as an argument so that one function can get data from all sheets

In [9]:
masterdf=pd.DataFrame()

In [196]:
def getdata2(file):
    dflist=[]
    coordlist=[]
    sheet=find1045(file)
    alphabet=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    found=False
    keywords1 = ["evaluation", "management", "services"]
    keywords2 = ["laboratory", "pathology", "services"]
    keywords3= ["radiology", "services"]
    keywords4 = ["medicine", "services"]
    keywords5 = ["surgery", "services"]
    keywords6 = ["other", "common", "outpatient", "procedures"]
    keywords7 = ["count", "of", "reported", "procedures",]
    keywordslists=[keywords1,keywords2,keywords3,keywords4,keywords5,keywords6,keywords7]
    for wordlist in keywordslists:
        for row in sheet.iter_rows():
                for cell in row:
                    heading = str(cell.value).lower()
                    wordsInHeading=heading.split()
                    match =  all(item in wordsInHeading for item in wordlist)
                    if match==True:
                        coord1=cell.coordinate
                        coordlist.append(coord1)
                        break
    #print(coordlist)
    for coordinate in coordlist[0:-1]:
        #print("coord")
        #print(coordinate)
        splitcoord=re.split('(\d+)', coordinate)
        nextbreak=coordlist[int(coordlist.index(coordinate)+1)]
        print("nextbreak")
        print(nextbreak)
        splitbreak=re.split('(\d+)', nextbreak)
        #print("splitbreak")
        #print(splitbreak)
        lastcell=f"{int(splitbreak[1])-1}"
        coord2=f"C{lastcell}"
        #print("coord2")
        #print(coord2)

        cellgroup=[coordinate,coord2]
        #print(cellgroup)
        df=pd.DataFrame(sheet[f"{cellgroup[0]}:{cellgroup[1]}"])

        dflist.append(df)

    return(dflist)

  
    

In [197]:
def getdata2comment(file):
    dflist=[]
    coordlist=[]
    sheet=find1045(file)
    print(sheet.max_row)
    alphabet=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    found=False
    keywords1 = ["evaluation", "management", "services"]
    keywords2 = ["laboratory", "pathology", "services"]
    keywords3= ["radiology", "services"]
    keywords4 = ["medicine", "services"]
    keywords5 = ["surgery", "services"]
    keywords6 = ["other","common"]
    keywords7 = ["count", "of", "reported", "procedures",]
    keywordslists=[keywords1,keywords2,keywords3,keywords4,keywords5,keywords6,keywords7]
    for wordlist in keywordslists:
        for row in sheet.iter_rows():
                for cell in row:
                    heading = str(cell.value).lower()
                    wordsInHeading=heading.split()
                    match =  all(item in wordsInHeading for item in wordlist)
                    if match==True:
                        coord1=cell.coordinate
                        coordlist.append(coord1)
                        break
    #print(coordlist)
    for coordinate in coordlist[0:-1]:
        #print("coord")
        #print(coordinate)
        splitcoord=re.split('(\d+)', coordinate)
        nextbreak=coordlist[int(coordlist.index(coordinate)+1)]
        print("nextbreak")
        print(nextbreak)
        splitbreak=re.split('(\d+)', nextbreak)
        #print("splitbreak")
        #print(splitbreak)
        lastcell=f"{int(splitbreak[1])-1}"
        coord2=f"C{lastcell}"
        #print("coord2")
        #print(coord2)

        cellgroup=[coordinate,coord2]
        #print(cellgroup)
        df=pd.DataFrame(sheet[f"{cellgroup[0]}:{cellgroup[1]}"])
    print(coordlist)

    return(dflist)

In [198]:
def convertvalue(x):
    return x.value

In [199]:
mylist=[1,2,3]
mylist[-1:1:-2]

[3]

In [200]:
#Get a list of all important dfs in the dataset, converted
convertlist=[]
masterlist=getdata2(xlsx_file)
for i in range(len(masterlist)):
    df=masterlist[i].applymap(convertvalue)
    convertlist.append(df)

nextbreak
A11
nextbreak
A25
nextbreak
A33
nextbreak
A40
nextbreak
A47
nextbreak
A56


In [201]:
for df in convertlist:
    if len(df)>0:
        df.columns = df.iloc[0]
        df["Service Type"]=df.iloc[0,0]
        df.rename(columns={ df.columns[0]: "Service" }, inplace = True)
        df=df.drop(df.index[0], inplace=True)

In [203]:
convertlist[5]

,Service,2019 CPT Code,Average Charge,Service Type
1,GLUCOSE BLOOD TEST,82962,165.4,Other Common Outpatient Procedures (list as ne...
2,"COLLECTION VENOUS BLOOD,VENIPUNCTURE",36415,16.55,Other Common Outpatient Procedures (list as ne...
3,NONINVASV OXYGEN SATUR;SINGLE,94760,89.3,Other Common Outpatient Procedures (list as ne...
4,BACTERIA CULTURE SCREEN,87081,172,Other Common Outpatient Procedures (list as ne...
5,None,None,None,Other Common Outpatient Procedures (list as ne...
6,None,None,None,Other Common Outpatient Procedures (list as ne...
7,None,None,None,Other Common Outpatient Procedures (list as ne...
8,None,None,None,Other Common Outpatient Procedures (list as ne...


In [ ]:
#Pull relavant data from worksheets

startFolder ="/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster"

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        hospital_df = pd.DataFrame(pd.read_excel(file))
        lines=f.readlines()
        f.close()
        f=open(file,'w')
        for line in lines:
            newline = "No you are not"
            f.write(newline)
        f.close()
        
        